This notebook tests the functionality of sensor/hue.py outside of Home-assistant, allowing the development of the unit tests

In [1]:
pwd

'/Users/robincole/Documents/Github/Hue-sensors-HASS'

In [2]:
from unittest.mock import Mock, MagicMock, patch
import unittest
import json
import pprint
import requests
import requests_mock
from datetime import datetime, timedelta
import hue_sensors as hs

def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

# for loading URL    
def load_url(filename):
    try:
        with open(filename, 'r') as fp:
            url = json.load(fp)
    except Exception as e:
        print('Failed to load url')
        url = None
    return url['url']

In [3]:
hs.__version__

1.1

In [4]:
filename = '/Users/robincole/Desktop/Hue_url.json'
url = load_url(filename)    # base URL of the remote, ending /sensors

In [5]:

class HueSensorData(object):
    """Get the latest sensor data."""

    def __init__(self, url, parse_hue_api_response):
        """Initialize the data object."""
        self.url = url
        self.data = None
        self.parse_hue_api_response = parse_hue_api_response

    # Update only once in scan interval.
  #  @Throttle(SCAN_INTERVAL)
    def update(self):
        """Get the latest data."""
        response = requests.get(self.url)
        if response.status_code != 200:
            _LOGGER.warning("Invalid response from API")
        else:
            self.data = self.parse_hue_api_response(response.json())


class HueSensor():  # Entity
    """Class to hold Hue Sensor basic info."""

    ICON = 'mdi:run-fast'

    def __init__(self, hue_id, data):
        """Initialize the sensor object."""
        self._hue_id = hue_id
        self._data = data    # data is in .data
        self._icon = None
        self._name = self._data.data[self._hue_id]['name']
        self._model = self._data.data[self._hue_id]['model']
        self._state = self._data.data[self._hue_id]['state']
        self._attributes = {}

    @property
    def name(self):
        """Return the name of the sensor."""
        return self._name

    @property
    def state(self):
        """Return the state of the sensor."""
        return self._state

    @property
    def icon(self):
        """Icon to use in the frontend, if any."""
        return self._icon

    @property
    def device_state_attributes(self):
        """Attributes."""
        return self._attributes

    def update(self):
        """Update the sensor."""
        self._data.update()
        self._state = self._data.data[self._hue_id]['state']
        if self._model == 'SML':
            self._icon = 'mdi:run-fast'
            self._attributes['light_level'] = self._data.data[
                self._hue_id]['light_level']
            self._attributes['battery'] = self._data.data[
                self._hue_id]['battery']
            self._attributes['last updated'] = self._data.data[
                self._hue_id]['last_updated']
            self._attributes['lux'] = self._data.data[
                self._hue_id]['lux']
            self._attributes['dark'] = self._data.data[
                self._hue_id]['dark']
            self._attributes['daylight'] = self._data.data[
                self._hue_id]['daylight']
            self._attributes['temperature'] = self._data.data[
                self._hue_id]['temperature']
        elif self._model == 'RWL':
            self._icon = 'mdi:remote'
            self._attributes['last updated'] = self._data.data[
                self._hue_id]['last_updated']
            self._attributes['battery'] = self._data.data[
                self._hue_id]['battery']
        elif self._model == 'ZGP':
            self._icon = 'mdi:remote'
            self._attributes['last updated'] = self._data.data[
                self._hue_id]['last_updated']
        elif self._model == 'Geofence':
            self._icon = 'mdi:cellphone'


In [6]:
# modified from sensors/hue.py 
def setup_platform():
    """Set up the Hue sensors."""
    #import hue_sensors as hs    
    #url = hass.data[DOMAIN] + '/sensors'
    data = HueSensorData(url, hs.parse_hue_api_response)
    data.update()
    sensors = []
    for key in data.data.keys():
        sensors.append(HueSensor(key, data))
    return sensors

In [7]:
sensors = setup_platform()
print("There are {} sensors".format(len(sensors)))

There are 7 sensors


In [8]:
for sensor in sensors:
    print(sensor.name)

Remote bedroom
Hall motion Sensor
Bedroom motion sensor
Robins iPhone
Living room motion sensor
Living room remote
Hall remote


In [9]:
for sensor in sensors:
    sensor.update()


s = sensors[5]
print(s.name)
print(s.state)
print(s.device_state_attributes)

Living room remote
1_click
{'last updated': ['2017-10-13', '05:34:33'], 'battery': 100}


## Tests
Need to capture the return of parse_hue_api_response and use this in test.

In [29]:
# Load the saved raw API data
with open('tests/hue_sensors.json', 'r') as fp:
    mock_data = json.load(fp)

In [30]:
print_json(hs.parse_hue_api_response(mock_data))

{'Geofence': {'model': 'GEO', 'name': 'Robins iPhone', 'state': 'on'},
 'RWL_1e-02': {'battery': 100,
               'last_updated': ['2017-10-13', '05:34:33'],
               'model': 'RWL',
               'name': 'Living room remote',
               'state': '1_click'},
 'RWL_9c-02': {'battery': 100,
               'last_updated': ['2017-10-13', '05:36:46'],
               'model': 'RWL',
               'name': 'Hall remote',
               'state': '1_click'},
 'RWL_dc-02': {'battery': 100,
               'last_updated': ['2017-10-13', '06:01:04'],
               'model': 'RWL',
               'name': 'Remote bedroom',
               'state': '2_hold'},
 'SML_28-02': {'battery': 100,
               'dark': True,
               'daylight': False,
               'last_updated': ['2017-10-13', '06:46:19'],
               'light_level': 0,
               'lux': 1.0,
               'model': 'SML',
               'name': 'Hall motion Sensor',
               'state': 'off',
               

Mock object to hold mock data

In [40]:
class MockHueSensorData():
    """Class to hold mock_data."""

    def __init__(self):
        """Initialize the mock object."""

In [41]:
mock_data_object = MockHueSensorData()

In [42]:
mock_data_object.data = hs.parse_hue_api_response(mock_data)

In [43]:
for key in mock_data_object.data.keys():
    print(key)

RWL_dc-02
SML_28-02
SML_ce-02
Geofence
SML_9c-02
RWL_1e-02
RWL_9c-02


In [48]:
class TestSensor(unittest.TestCase):
    
    def test_setup_platform(self):
        """Set up the Hue sensors."""
    
        sensors = []
        for key in mock_data_object.data.keys():
            sensors.append(HueSensor(key, mock_data_object))

        bedroom_remote = sensors[0]
        assert bedroom_remote.name == 'Remote bedroom'
        assert bedroom_remote.state == '2_hold'
        
        hall_motion_sensor = sensors[1]
        assert hall_motion_sensor.name == 'Hall motion Sensor'

In [49]:
# Run the unit test (and don't shut down the Jupyter Notebook)
unittest.main(argv=['ignored', '-v'], exit=False)

test_setup_platform (__main__.TestSensor)
Set up the Hue sensors. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
